In [7]:
import nest_asyncio
nest_asyncio.apply()

In [8]:
from langchain.tools import tool
from duckduckgo_search import DDGS
import wikipedia
import asyncio
from playwright.async_api import async_playwright

@tool
def search_wikipedia(query: str) -> str:
    """Searches Wikipedia for a summary of the query."""
    try:
        return wikipedia.summary(query)
    except:
        return "Wikipedia search failed."

@tool
def duckduckgo_top_url(query: str) -> str:
    """Returns the first DuckDuckGo result URL for the query."""
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=5))
        return results[0]['href'] if results else "No result found."

@tool
def scrape_page(url: str) -> str:
    """Scrapes <p> text from a web page using Playwright."""
    async def scrape():
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.goto(url)
            paragraphs = await page.locator("p").all_inner_texts()
            await browser.close()
            return "\n".join(paragraphs)

    return asyncio.run(scrape())

In [14]:
from langchain_ollama import OllamaLLM
from langchain.agents import initialize_agent, AgentType

llm = OllamaLLM(
    model="llama3",
    system="You are a cybersecurity research assistant. Only retrieve and summarize public, reputable sources such as Wikipedia, news sites, and academic resources."
)

In [15]:
from langchain.agents import Tool

tools = [search_wikipedia, duckduckgo_top_url, scrape_page]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

In [ ]:
response = agent.run("Conduct a general cybersecurity literature review of the recent XZ utility incident. Include sources from Wikipedia or online articles. Use scraping if needed.")
print(response)



> Entering new AgentExecutor chain...
A great starting point! Let's begin our journey to conduct a general cybersecurity literature review on the recent XZ utility incident.

Thought: To get started, I should search for relevant information on Wikipedia and then check if there are any online articles that provide more insights. I'll also use scraping if necessary to gather more information.

Action: search_wikipedia
Action Input: "XZ utility incident"
Observation: In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution through OpenSSH on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux

In [ ]:
with open("xz_backdoor_research.txt", "w") as f:
    f.write(response)